**1. マルコフ性とは**
> マルコフ性とは, システムの全体を見るのではなく, 近傍を見て, 未来を予測すること. 天気予報で言うと, (過去のすべての履歴を使用するのではなく) 今日の気象条件のみを使用して, 明日の天気を予想するときに機能する.

**2. マルコフモデル**
> マルコフモデルは, 複数の状態が存在する際に, その状態から次の状態へ遷移する確率を示したモデルのことをいう. 時刻に関して視覚した表現方法を**トレリス線図**という. 遷移行列:T(=3×3)と, 各状態のスコア:s(=3×1)とすると, tf.matmul(T,s)で,各状態からの遷移の期待値が求まる.


**3. 隠れマルコフモデル**
> 真の状態(晴れ, 雨など)を直接観測できないとき, **隠れ状態**をいう. 隠れ状態を使うマルコフモデルが, 隠れマルコフモデルと呼ばれる.

> 隠れマルコフモデルでは, 隠れ状態は, 可能性のある測定可能な観測値を出力する(晴れた状態では, 気温が30℃など). その出力確率を定義する必要がある. 出力確率は, N(状態数(晴れ, 雨など))×M(観測値からわかる観測タイプ(暑い, 寒いなど))の行列である.

> そして, 初期確率を定義する. 



**4. 前向きアルゴリズム**
> 観測の確率を計算する. その際に, すべての可能性を列挙すると指数関数的な時間がかかるので, **動的プログラミング**を用いる. 動的プログラミングは, 問題を分割し, 参照テーブルに結果を保存していく方法である.



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd "/content/drive/My Drive/Colab Notebooks/春休み課題"

/content/drive/My Drive/Colab Notebooks/春休み課題


In [0]:
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


**HMMクラス**: 隠れマルコフモデルパラメータを取得する

In [0]:
class HMM(object):
  pass

In [0]:
  def __init__(self, initial_prob, trans_prob, obs_prob):
    self.N = np.size(initial_prob)    # 状態数
    self.initial_prob = initial_prob  # 初期確率ベクトル
    self.trans_prob = trans_prob      # 遷移確率行列
    self.emission = tf.constant(obs_prob)  # 出力確率行列

    # サイズの確認
    assert self.initial_prob.shape == (self.N, 1)
    assert self.trans_prob.shape == (self.N, self.N)
    assert obs_prob.shape[0] == self.N

    # 前向きアルゴリズムに使うプレースホルダー
    self.obs_idx = tf.placeholder(tf.int32) # 入力:観測値('walk')とか
    self.fwd = tf.placeholder(tf.float64) # 出力:周辺同時確率{('Sunny'⋀'walk')⋁('Rainy'⋀'walk')}とか

    # Viterbiキャッシュ
    self.viterbi = tf.placeholder(tf.float64)
  
  HMM.__init__ = __init__ 

**クイックヘルパー関数**: 任意の行列から, データを取得(スライス)する

In [0]:
  def get_emission(self, obs_idx):
    slice_location = [0, obs_idx]  # スライスする場所
    num_rows = tf.shape(self.emission)[0] # 出力確率行列の行数=隠れ状態数
    slice_shape = [num_rows, 1]  # スライスする行のサイズ
    return tf.slice(self.emission, slice_location, slice_shape)  # 出力確率行列の指定列を1列分取り出す
        #  tf.slice(対象のテンソル, 開始位置, スライスサイズ)

  HMM.get_emission = get_emission

**キャッシュの初期化**: 前向きアルゴリズムにおける動的プログラミングの途中結果を保存するキャッシュを作る

In [0]:
  def forward_init_op(self):
    obs_prob = self.get_emission(self.obs_idx) # 出力確率行列の指定列を取り出して, #(2,1)
    fwd = tf.multiply(self.initial_prob, obs_prob) # 初期確率と要素ごとにかけていく.
    return fwd  #(2,1)
  HMM.forward_init_op = forward_init_op

**キャッシュの更新**: 観測時ごとにキャッシュを更新 = **前向きステップの実行**

In [0]:
  def forward_op(self):
    transitions = tf.matmul(self.fwd, tf.transpose(self.get_emission(self.obs_idx))) #(2,1)×(1,2)=(2,2)
      #遷移 : 前の観測状態と,今回の観測状態の同時確率
    weighted_transitions = transitions * self.trans_prob #(2,2)×(2,2)=(2,2)
      #遷移の重み : それぞれの隠れ状態におけるこれまでの観測状態が当てはまる確率
    fwd = tf.reduce_sum(weighted_transitions, axis=0) #(1,2)
      #それらの列和 : その観測状態になる確率
    fwd_reshaped = tf.reshape(fwd, tf.shape(self.fwd)) #(2,1)
    return fwd_reshaped
  HMM.forward_op = forward_op

**Viterbiアルゴリズム(前向きキャッシュを更新する操作)**

In [0]:
  def decode_op(self):
    transitions = tf.matmul(self.viterbi, tf.transpose(self.get_emission(self.obs_idx))) #(2,2)
    weighted_transitions = transitions * self.trans_prob
    viterbi = tf.reduce_max(weighted_transitions, axis=0) #(1,2)
    viterbi_reshaped = tf.reshape(viterbi, tf.shape(self.viterbi))
    return viterbi_reshaped
  HMM.decode_op = decode_op

** ビタビ符号のバックポインタを更新**

In [0]:
  def backpt_op(self):
    back_transitions = tf.matmul(self.viterbi, np.ones((1, self.N))) #(2,2)
    weighted_back_transitions = back_transitions * self.trans_prob
    value = tf.argmax(weighted_back_transitions, axis=0) #(1,2)
    return value
  HMM.backpt_op = backpt_op

**ビタビ符号関数**

In [0]:
def viterbi_decode(sess, hmm, observations):
  viterbi = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx: observations[0]}) #(2,1)
  backpts = np.ones((hmm.N, len(observations)), 'int32') * -1  # (2,5)

  for t in range(1, len(observations)):
    viterbi, backpt = sess.run([hmm.decode_op(), hmm.backpt_op()], feed_dict={hmm.obs_idx: observations[t], hmm.viterbi: viterbi})
    print('backpt: {}'.format(backpt))
    print('backpt.shape: {}'.format(tf.shape(backpt)))
    backpts[:, t] = backpt #(1,2)
  tokens = [viterbi[:, -1].argmax()]

  for i in range(len(observations)-1, 0, -1):
    tokens.append(backpts[tokens[-1], i])
  return tokens[::-1]

**前向きアルゴリズムの定義**

In [0]:
def forward_algorithm(sess, hmm, observations):
  fwd = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx:observations[0]}) # キャッシュの初期化:(2,1)
  for t in range(1, len(observations)):
    fwd = sess.run(hmm.forward_op(), feed_dict={hmm.obs_idx: observations[t], hmm.fwd: fwd})
    print('t: {}, fwd: {}'.format(t, fwd))
  prob = sess.run(tf.reduce_sum(fwd))
  return prob  #観測確率

**HMMの定義**
1.   初期確率ベクトル - 状態が生じるはじめの確率
2.   遷移確率行列 - 現在の状態が与えられてたとき, 次の状態への移行する確率
3.   出力確率行列 - その状態から生じるそれぞれの観測タイプの確率



```
states = ('Rainy', 'Sunny')
observations = ('walk', 'shop', 'clean')

initial_probability = ('Rainy':0.6, 'Sunny':0.4)
transition_probability = {
    'Rainy' : {'Rainy':0.7, 'Sunny':0.3},
    'Sunny' : {'Rainy':0.4, 'Sunny':0.6}
}
emission_probability = {
    'Rainy' : {'walk':0.1, 'shop':0.4, 'clean':0.5},
    'Sunny' : {'walk':0.6, 'shop':0.3, 'clean':0.1}
}
```



In [0]:
if __name__ == '__main__':
  initial_prob = np.array([[0.6],
                           [0.4]])
  trans_prob = np.array([[0.7, 0.3],
                         [0.4, 0.6]])
  obs_prob = np.array([[0.1, 0.4, 0.5],
                       [0.6, 0.3, 0.1]])
  
  hmm = HMM(initial_prob, trans_prob, obs_prob)

  observations = [0, 1, 1, 2, 1]  # 観測したのは,時刻ごとに, 'walk','shop','shop','clean','shop'

  with tf.Session() as sess:
    #prob = forward_algorithm(sess, hmm, observations)
    #print('Probability of observing {} is {}'.format(observations, prob))
    
    seq = viterbi_decode(sess, hmm, observations)
    print('Most likely hidden states are {}'.format(seq))

backpt: [1 1]
backpt.shape: Tensor("Shape_24:0", shape=(1,), dtype=int32)
backpt: [0 1]
backpt.shape: Tensor("Shape_27:0", shape=(1,), dtype=int32)
backpt: [0 1]
backpt.shape: Tensor("Shape_30:0", shape=(1,), dtype=int32)
backpt: [0 0]
backpt.shape: Tensor("Shape_33:0", shape=(1,), dtype=int32)
Most likely hidden states are [1, 0, 0, 0, 0]
